# Import packages

In [1]:
import collections
import copy
import json
import multiprocessing
import math
import os
import random
import shutil
import subprocess

import aiohttp
import tqdm

import pandas as pd

In [2]:
DATA_REPOS = 'data/repos/'
DATA_JS_FILES = 'data/js_files/'
DATA_FUNCS = 'data.files/funcs/'
DATA_OBFUSCATED = 'data.files/obfuscated/'
DATA_AST_FEATURES = 'data.files/ast_features/'
DATA_FEATURES_JSON = 'data.files/features.json'

In [3]:
def create_dir_if_not_exists(dir_: str):
    try:
        os.mkdir(dir_)
    except Exception as e:
        print(e)

# Extract functions from js files

__!NB: You should have [node.js](https://nodejs.org/en/) and [esprima](https://www.npmjs.com/package/esprima) npm package installed__

In [4]:
from sklearn.model_selection import train_test_split


js_files = sorted([os.path.join(path, name) for path, _, files in os.walk(DATA_JS_FILES) for name in files])


files_to_funcs_sample, files_sample = train_test_split(js_files, test_size=0.3, random_state=11)

print('Files to funcs sample size: ', len(files_to_funcs_sample))
print('Files sample  size: ', len(files_sample))

Files to funcs sample size:  116290
Files sample  size:  49839


Create `data/funcs/` directory

In [5]:
create_dir_if_not_exists(DATA_FUNCS)

[Errno 17] File exists: 'data.files/funcs/'


Extract functions from JS files

In [11]:
def process_file(file):
    input_file = file
    output_file = os.path.join(DATA_FUNCS, file.lstrip(DATA_JS_FILES))

    command = [
        'node',
        'scripts/parse_functions.js',
        input_file,
        output_file
    ]

    subprocess.run(command)


with multiprocessing.Pool(processes=os.cpu_count() - 1) as pool:
    for _ in tqdm.tqdm(pool.imap_unordered(process_file, files_to_funcs_sample), 
                       total=len(files_to_funcs_sample)):
        pass


116290it [1:27:59, 22.03it/s]


# Generate obfuscated samples

Create `data.files/obfuscated/` directory with subdirectories

In [6]:
create_dir_if_not_exists(DATA_OBFUSCATED)

[Errno 17] File exists: 'data.files/obfuscated/'


In [7]:
OBFUSCATORS = ['defendjs', 'jscrewit', 'gnirts', 'javascript-obfuscator',
               'jsobfu', 'jfogs', 'closure', 'uglifyjs', 'original']


for obfuscator in OBFUSCATORS:
    create_dir_if_not_exists(os.path.join(DATA_OBFUSCATED, obfuscator))


[Errno 17] File exists: 'data.files/obfuscated/defendjs'
[Errno 17] File exists: 'data.files/obfuscated/jscrewit'
[Errno 17] File exists: 'data.files/obfuscated/gnirts'
[Errno 17] File exists: 'data.files/obfuscated/javascript-obfuscator'
[Errno 17] File exists: 'data.files/obfuscated/jsobfu'
[Errno 17] File exists: 'data.files/obfuscated/jfogs'
[Errno 17] File exists: 'data.files/obfuscated/closure'
[Errno 17] File exists: 'data.files/obfuscated/uglifyjs'
[Errno 17] File exists: 'data.files/obfuscated/original'


Randomly assign obfuscator to function

In [8]:
random.seed(144)


samples = sorted([os.path.join(path, name) for path, _, files in os.walk(DATA_FUNCS) for name in files])
samples += files_sample

random.shuffle(samples)
print('Samples: ', len(samples))

Samples:  585757


In [12]:
OBFUSCATOR_TO_FUNCTIONS = collections.defaultdict(list)


for i, obfuscator in enumerate(OBFUSCATORS):
    OBFUSCATOR_TO_FUNCTIONS[obfuscator] += samples[
        math.floor(len(samples) / 11 * i) : math.floor(len(samples) / 11 * (i + 1))
    ]

OBFUSCATOR_TO_FUNCTIONS['original'] += samples[math.floor(len(samples) / 11 * (i + 1)):]


print('Functions sampling results:')
for k, v in OBFUSCATOR_TO_FUNCTIONS.items():
    print(f'{k}: {len(v)}')


Functions sampling results:
defendjs: 53250
jscrewit: 53251
gnirts: 53250
javascript-obfuscator: 53251
jsobfu: 53251
jfogs: 53250
closure: 53251
uglifyjs: 53251
original: 159752


Generate obfuscated samples

In [13]:
for obfuscator in OBFUSCATORS:
    def process_file(file):
        input_file = os.path.join(os.getcwd(), file)
        
        stripped_file = file.lstrip(DATA_FUNCS) if DATA_FUNCS in file else file.lstrip(DATA_JS_FILES)
        output_file = os.path.join(os.getcwd(), DATA_OBFUSCATED, obfuscator, stripped_file)

        command = ['bash', f'run_{obfuscator}.sh', input_file, output_file]
        subprocess.run(command, cwd='scripts/obfuscate/')

    def run_obfuscator():
        with multiprocessing.Pool(processes=os.cpu_count() - 1) as pool:
            for _ in tqdm.tqdm(pool.imap_unordered(process_file, OBFUSCATOR_TO_FUNCTIONS[obfuscator]),
                               total=len(OBFUSCATOR_TO_FUNCTIONS[obfuscator])):
                if obfuscator != 'original' and \
                        len(os.listdir(os.path.join(DATA_OBFUSCATED, obfuscator))) > 20000:
                    return
    
    print(obfuscator)
    run_obfuscator()

defendjs


 40%|███▉      | 21073/53250 [2:58:53<4:33:08,  1.96it/s] 

jscrewit



 38%|███▊      | 20001/53251 [58:57<1:38:00,  5.65it/s]

gnirts


...

 39%|███▉      | 20673/53251 [17:50:19<28:06:41,  3.11s/it]

uglifyjs



 37%|███▋      | 19969/53251 [41:50<1:09:45,  7.95it/s]


original


100%|██████████| 159752/159752 [17:09<00:00, 155.25it/s]


In [ ]:
# create_dir_if_not_exists(os.path.join(DATA_OBFUSCATED_FUNCS, 'original'))

# for file in functions:
#     src = file
#     dst = os.path.join(DATA_OBFUSCATED_FUNCS, 'original', file.lstrip(DATA_FUNCS))
#     shutil.copyfile(src, dst)

Make `data.files/obfuscated/defendjs` data structure complient with the rest of the directories

In [14]:
defendjs_samples = sorted(
    [os.path.join(path, name) 
     for path, _, files in os.walk(os.path.join(DATA_OBFUSCATED, 'defendjs'))
     for name in files]
)


print('Found defendjs files: ', len(defendjs_samples))

Found defendjs files:  20004


In [15]:
for file in tqdm.tqdm(defendjs_samples):
    src = file
    dst = os.path.join(os.path.dirname(os.path.dirname(file)), os.path.basename(file))

    try:
        if os.path.join(DATA_OBFUSCATED, 'defendjs') not in dst:
            continue

        with open(src, 'rb') as fr:
            content = fr.read()

        shutil.rmtree(os.path.dirname(src))

        with open(dst, 'wb') as fw:
            fw.write(content)

    except Exception as e:
        print(e)


100%|██████████| 20004/20004 [00:27<00:00, 726.76it/s]


In [16]:
defendjs_samples = sorted(
    [os.path.join(path, name) 
     for path, _, files in os.walk(os.path.join(DATA_OBFUSCATED, 'defendjs'), topdown=False)
     for name in files]
)

print('Moved defendjs files: ', len(defendjs_samples))

Moved defendjs files:  20004


# Extract features

In [17]:
create_dir_if_not_exists(DATA_AST_FEATURES)

In [18]:
for obfuscator in OBFUSCATORS:
    create_dir_if_not_exists(os.path.join(DATA_AST_FEATURES, obfuscator))

In [19]:
import sys
sys.path.extend(['/Users/grigory/Documents/2019/PyProj/ast_to_neo4j/'])


from converter import converter


obfuscated_functions = sorted(
    [os.path.join(path, name) for path, _, files in os.walk(DATA_OBFUSCATED) for name in files]
)


def process_file(file):
    input_file = file
    obfusactor = os.path.dirname(input_file).split('/')[-1]
    output_file = os.path.join(DATA_AST_FEATURES, obfusactor, file.lstrip(DATA_OBFUSCATED).replace('/', '_'))

    try:
        features_extractor = converter.AstToCsvConverter()
        features_extractor.parse(path=input_file)
        features_extractor.convert()
        features_extractor.save_features(path=output_file)
 
    except Exception as e:
        print(e)


with multiprocessing.Pool(processes=os.cpu_count() - 1) as pool:
    for _ in tqdm.tqdm(pool.imap_unordered(process_file, obfuscated_functions), total=len(obfuscated_functions)):
        pass

    

  0%|          | 1/319771 [00:00<44:12:08,  2.01it/s]

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


  7%|▋         | 21081/319771 [27:49<6:31:36, 12.71it/s] 

Expecting value: line 1 column 1 (char 0)


  7%|▋         | 21528/319771 [28:40<6:50:20, 12.11it/s] 

...

 99%|█████████▉| 317686/319771 [12:27:00<02:09, 16.14it/s]

Expecting value: line 1 column 1 (char 0)


 99%|█████████▉| 317896/319771 [12:27:18<03:08,  9.96it/s]

Expecting value: line 1 column 1 (char 0)


100%|██████████| 319771/319771 [13:25:47<00:00,  6.61it/s] 


In [20]:
ast_features = sorted(
    [os.path.join(path, name) for path, _, files in os.walk(DATA_AST_FEATURES) for name in files]
)


len(ast_features)

315803

# Create train and test dataframes

Create train and test samples

In [21]:
from sklearn.model_selection import train_test_split


train_ast, test_ast = train_test_split(ast_features, test_size=0.3, random_state=11)
train_ast, val_ast = train_test_split(train_ast, test_size=0.4, random_state=12)

print('Train size: ', len(train_ast))
print('Test size: ', len(test_ast))
print('Val size: ', len(val_ast))

Train size:  132637
Test size:  94741
Val size:  88425


In [22]:
# iterate over train and get node names and node pairs


FEATURES = set()

for file_path in tqdm.tqdm(train_ast):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        for k_first in data.keys():
            for k_second in data[k_first].keys():
                FEATURES.add(f'{k_first};{k_second}')
    except Exception as e:
        print(e)


with open(DATA_FEATURES_JSON, 'wb') as f:
    f.write(json.dumps(sorted(list(FEATURES))).encode('utf-8'))

100%|██████████| 132637/132637 [01:51<00:00, 1194.13it/s]


In [23]:
with open(DATA_FEATURES_JSON, 'rb') as f:
    FEATURES = json.loads(f.read().decode())


FEATURES = sorted([tuple(f.split(';')) for f in FEATURES])
print('Collected features count: ', len(FEATURES))

Collected features count:  892


Collect train dataset

In [24]:
def collect_dataframe(ast):
    dataframe_rows = []

    for file in tqdm.tqdm(ast):
        try:
            with open(file, 'r') as f:
                data = json.load(f)

            obfuscator = os.path.dirname(file).split('/')[-1]
            feature_vector = {'file': file, 'obfuscator': obfuscator}

            for key_first, key_second in FEATURES:
                feature_vector[f'{key_first};{key_second}'] = data[key_first].get(key_second, 0)

            dataframe_rows.append(feature_vector)

        except Exception as e:
            print(e)

    return pd.DataFrame(dataframe_rows)

In [25]:
train_df = collect_dataframe(train_ast)

100%|██████████| 132637/132637 [02:39<00:00, 831.03it/s]


In [26]:
pair_features = [f for f in train_df.columns if 'node_pairs;' in f]
single_features = [f for f in train_df.columns if 'nodes;' in f]
other_features = [f for f in train_df.columns if f not in pair_features + single_features]

In [27]:
def supplement_features(df: pd.DataFrame):
    df[single_features] = df[single_features].astype(int)
    df[pair_features] = df[pair_features].astype(int)

    df['sum'] = df[single_features].sum(axis=1)
    df = df[df['sum'] >= 25].copy(deep=True)
    del df['sum']

    df['v_types'] = df[single_features].astype(bool).sum(axis=1) / len(single_features)
    df['e_types'] = df[pair_features].astype(bool).sum(axis=1) / len(pair_features)

    df[single_features] = (df[single_features].to_numpy().T / df[single_features].to_numpy().sum(axis=1)).T
    df[pair_features] = (df[pair_features].to_numpy().T / df[pair_features].to_numpy().sum(axis=1)).T

    return df


In [28]:
train_df = supplement_features(train_df)
print('Train dataset: ', train_df.shape)

Train dataset:  (106469, 896)


Save train dataframe to `data/train.csv`

In [29]:
train_df.to_csv('data.files/train.csv', index=False)

Collect test dataframe

In [30]:
test_df = collect_dataframe(test_ast)

100%|██████████| 94741/94741 [02:13<00:00, 711.51it/s]


In [31]:
test_df = supplement_features(test_df)
print('Test dataset: ', test_df.shape)

Test dataset:  (75856, 896)


In [32]:
test_df.to_csv('data.files/test.csv', index=False)

Collect validation dataframe

In [33]:
val_df = collect_dataframe(val_ast)

100%|██████████| 88425/88425 [01:51<00:00, 791.94it/s]


In [34]:
val_df = supplement_features(val_df)
print('Validation dataset: ', val_df.shape)

Validation dataset:  (71058, 896)


In [35]:
val_df.to_csv('data.files/val.csv', index=False)